In [1]:
import os, json
import pandas as pd

# this finds our json files
path_to_json = 'F:/kalu/BITS/DeepLearning/assignment1/CORD/train/json/'
path_to_json = 'F:/kalu/BITS/DeepLearning/assignment1/CORD/test/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['validline'])
f = open("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv", "a")

f = open("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv", "a")
#print("filename",",","index",",","x1",",",'x2',",",'x3',",",'x4',",",'y1',",",'y2',",",'y3',",",'y4',",",'is_key',",",'row_id',",",'text',",",'category',",","group_id",file=f)


# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        
       # print(json_text)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        lines = json_text['valid_line']
        
        for x in lines:
            words = x['words']
            category = x['category']
            group_id = x['group_id']
            for y in words:
                print(js,index,y['quad']['x1'],y['quad']['x2'],y['quad']['x3'],y['quad']['x4'],y['quad']['y1'],y['quad']['y2'],y['quad']['y3'],y['quad']['y4'],y['is_key'],y['row_id'],"\""+y['text']+"\"",category,group_id,file=f,sep=",")

       # city = json_text['features'][0]['properties']['name']
        #lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        #f.close()
        #jsons_data.loc[index] = [words, city, lonlat]

# now that we have the pertinent json data in our DataFrame let's look at it
f.close()

In [2]:
import pandas as pd
receipts_df = pd.read_csv("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv")
receipts_df.columns = receipts_df.columns.str.strip()
receipts_df['text'] = receipts_df['text'].str.strip()


In [3]:
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [4]:
char2idx = {"START":0,"END":1,"a":2,"b":3,"c":4,"d":5,"e":6,"f":7,"g":8,"h":9,"i":10,"j":11,"k":12,"l":13,"m":14,"n":15,"o":16,"p":17,"q":18,"r":19,"s":20,"t":21,"u":22,"v":23,"w":24,"x":25,"y":26,"z":27,"0":28,"1":29,"2":30,"3":31,"4":32,"5":33,"6":34,"7":35,"8":36,"9":37,".":38,"-":39,",":40,":":41," ":42}

In [5]:
#p_test.SentimentText=p_test.SentimentText.astype(str)
receipts_df.text = receipts_df.text.astype(str)
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [6]:
words = receipts_df['text']

In [7]:
indexed_words = []
j=43
for word in words:
    indexed_word = []
    #print(word)
    word = word.lower();
    for i in range(len(word)):
        token = word[i]
        if token not in char2idx:
            char2idx[token] = j
            j += 1
        indexed_word.append(char2idx[token])
    indexed_words.append(indexed_word)

receipts_df["indexed_words"] = indexed_words;

In [8]:
import numpy as np
def get_bigram_probs(words, V, start_idx, end_idx, smoothing=1):
  # structure of bigram probability matrix will be:
  # (last word, current word) --> probability
  # we will use add-1 smoothing
  # note: we'll always ignore this from the END token
  bigram_probs = np.ones((V, V)) * smoothing
  for word in words:
    for i in range(len(word)):
      
      if i == 0:
        # beginning word
        word[i]
        bigram_probs[start_idx, word[i]] += 1
      else:
        # middle word
        bigram_probs[word[i-1], word[i]] += 1

      # if we're at the final word
      # we update the bigram for last -> current
      # AND current -> END token
      if i == len(word) - 1:
        # final word
        bigram_probs[word[i], end_idx] += 1

  # normalize the counts along the rows to get probabilities
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
  return bigram_probs

In [9]:
start_idx = char2idx['START']
end_idx = char2idx['END']
V = len(char2idx)



In [10]:
bigram_probs = get_bigram_probs(indexed_words, V, start_idx, end_idx, smoothing=0.1)

In [11]:
bigram_probs

array([[4.22357940e-06, 4.22357940e-06, 1.55047600e-02, ...,
        8.86951674e-05, 8.86951674e-05, 8.86951674e-05],
       [1.51515152e-02, 1.51515152e-02, 1.51515152e-02, ...,
        1.51515152e-02, 1.51515152e-02, 1.51515152e-02],
       [1.26412661e-05, 7.75036028e-02, 6.44704574e-04, ...,
        1.26412661e-05, 1.26412661e-05, 1.26412661e-05],
       ...,
       [1.16279070e-02, 2.44186047e-01, 1.16279070e-02, ...,
        1.16279070e-02, 1.16279070e-02, 1.16279070e-02],
       [1.16279070e-02, 1.16279070e-02, 1.27906977e-01, ...,
        1.16279070e-02, 1.16279070e-02, 1.16279070e-02],
       [1.16279070e-02, 2.44186047e-01, 1.16279070e-02, ...,
        1.16279070e-02, 1.16279070e-02, 1.16279070e-02]])

In [12]:
 # a function to calculate normalized log prob score
  # for a sentence
def get_score(word):
    #print("word")
    #print(word)
    score = 0
    if len(word) > 0 :
        for i in range(len(word)):
          if i == 0:
            # beginning word
            score += np.log(bigram_probs[start_idx, word[i]])
          else:
            # middle word
            score += np.log(bigram_probs[word[i-1], word[i]])
        # final word
        score += np.log(bigram_probs[word[i-1], end_idx])

    # normalize the score
    # return score / (len(word) + 1)
    if score == 0:
        return score;
    else :
        return  ((-1)*(len(word) + 1)/score)*100

In [13]:
from future.utils import iteritems

idx2char = dict((v, k) for k, v in iteritems(char2idx))
def get_word(word):
    return ''.join(idx2char[i] for i in word)

In [14]:
receipts_df['text_score'] = receipts_df.apply(lambda row : get_score(row['indexed_words']), axis = 1)

In [15]:
receipts_df

,filename,index,x1,x2,x3,x4,y1,y2,y3,y4,is_key,row_id,text,category,group_id,indexed_words,text_score
0,receipt_00000.json,0,256,270,270,256,374,374,390,390,0,2179893,x,menu.cnt,3,[25],39.130296
1,receipt_00000.json,0,258,270,270,258,402,402,418,418,0,2179894,x,menu.cnt,4,[25],39.130296
2,receipt_00000.json,0,258,272,272,258,428,428,444,444,0,2179895,x,menu.cnt,5,[25],39.130296
3,receipt_00000.json,0,260,274,274,260,456,456,470,470,0,2179896,x,menu.cnt,6,[25],39.130296
4,receipt_00000.json,0,258,274,274,258,480,480,496,496,0,2179897,x,menu.cnt,7,[25],39.130296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23665,receipt_00099.json,99,224,275,277,226,1064,1060,1086,1089,1,2242955,NON,total.creditcardprice,5,"[15, 16, 15]",29.891933
23666,receipt_00099.json,99,276,338,338,276,1060,1060,1086,1086,1,2242955,TUNAI,total.creditcardprice,5,"[21, 22, 15, 2, 10]",36.201937
23667,receipt_00099.json,99,450,515,513,449,1053,1056,1083,1080,0,2242955,"75,000",total.creditcardprice,5,"[35, 33, 40, 28, 28, 28]",60.294739
23668,receipt_00099.json,99,224,313,315,226,1090,1086,1112,1117,1,2242956,KEMBALI,total.changeprice,5,"[12, 6, 14, 3, 2, 13, 10]",49.614100


In [16]:
categories = receipts_df['category'] 
category2idx = {}
idx2category = {}
j=0;
for category in categories:
    if category not in category2idx:
        category2idx[category] = j
        idx2category[j] = category
        j+=1

category2idx

def get_index_for_category(category):
    return category2idx[category]


def get_category_for_index(index):
    return idx2category[index]
        

In [17]:
receipts_df['category_idx'] = receipts_df.apply(lambda row : get_index_for_category(row['category']), axis = 1)

In [18]:
data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index']]
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = data.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
print(x_scaled.shape)
X = pd.DataFrame(x_scaled,columns=['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index'])
X
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

def getcosine(X,Y):
    return cosine_similarity(X,Y)[0,0];

def getneighourscore(index,X):
    return X['text_score'].tolist()[0];

X["n1_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_cd done")
X["n2_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_cd done")
X["n3_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_cd done")
X["n4_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,4]]]), axis = 1)
print("n4_cd done")
X["n5_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,5]]]), axis = 1)
print("n5_cd done")

X["n1_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_ed done")
X["n2_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_ed done")
X["n3_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_ed done")
X["n4_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,4]]]), axis = 1)
print("n3_ed done")
X["n5_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,5]]]), axis = 1)
print("n5_ed done")

X["n1_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_score done")
X["n2_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_score done")
X["n3_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_score done")
X["n4_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,4]]]), axis = 1)
print("n4_score done")
X["n5_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,5]]]), axis = 1)
print("n5_score done")

X

(23670, 12)
n1_cd done
n2_cd done
n3_cd done
n4_cd done
n5_cd done
n1_ed done
n2_ed done
n3_ed done
n3_ed done
n5_ed done
n1_score done
n2_score done
n3_score done
n4_score done
n5_score done


,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score,...,n1_ed,n2_ed,n3_ed,n4_ed,n5_ed,n1_score,n2_score,n3_score,n4_score,n5_score
0,-0.448185,-0.618908,-0.619223,-0.448084,-0.914594,-0.914259,-0.925631,-0.925978,-0.572895,-0.598368,...,0.095663,0.184230,0.251243,0.259509,0.282882,-0.598368,-0.598368,-0.638298,-0.638298,-0.522912
1,-0.442199,-0.618908,-0.619223,-0.442097,-0.866315,-0.865981,-0.878641,-0.878992,-0.572895,-0.598368,...,0.088807,0.095908,0.181362,0.273679,0.277943,-0.598368,-0.598368,-0.598368,-0.638298,-0.638298
2,-0.442199,-0.613557,-0.613872,-0.442097,-0.821485,-0.821152,-0.835008,-0.835362,-0.572895,-0.598368,...,0.088807,0.092785,0.177172,0.218607,0.269875,-0.598368,-0.598368,-0.598368,-0.598368,-0.598368
3,-0.436213,-0.608205,-0.608522,-0.436110,-0.773206,-0.772874,-0.791374,-0.791732,-0.572895,-0.598368,...,0.085471,0.092785,0.173721,0.181523,0.264188,-0.598368,-0.598368,-0.598368,-0.598368,-0.686272
4,-0.442199,-0.608205,-0.608522,-0.442097,-0.731825,-0.731493,-0.747741,-0.748102,-0.572895,-0.598368,...,0.085471,0.089274,0.200659,0.178722,0.180936,-0.598368,-0.598368,-0.686272,-0.598368,-0.598368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23665,-0.543962,-0.605529,-0.600495,-0.537887,0.275128,0.268544,0.242399,0.246993,1.745522,-1.196121,...,0.000000,0.378303,0.487381,0.532358,0.504459,-1.196121,-1.300113,-1.291323,-0.822781,-0.812045
23666,-0.388324,-0.436956,-0.437296,-0.388215,0.268231,0.268544,0.242399,0.241958,1.745522,-0.787842,...,0.000000,0.390136,0.521709,0.446623,0.560031,-0.787842,-0.739529,-0.822781,-0.822781,-0.822781
23667,0.132466,0.036657,0.030900,0.129650,0.256161,0.261647,0.237365,0.231890,-0.572895,0.771043,...,0.000000,0.473100,0.473100,0.503625,0.506575,0.771043,0.495102,0.495102,0.756416,0.756416
23668,-0.543962,-0.503850,-0.498830,-0.537887,0.319958,0.313373,0.286033,0.293979,1.745522,0.079970,...,0.000000,0.425294,0.455111,0.579771,0.609805,0.079970,0.173901,0.173901,0.173901,0.173901


In [19]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers


In [20]:
#data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx']]
data = X

data['category_idx']=receipts_df['category_idx'];

dnnModel = models.Sequential()
# Layer 1 = input layer
# specify the input size in the first layer.

dnnModel.add(layers.Dense(60, activation='elu', input_shape= (26,),    kernel_regularizer=regularizers.l1_l2(l1=.01, l2=.01),
    bias_regularizer=regularizers.l2(.01),
    activity_regularizer=regularizers.l2(.01)))

# Layer 2 = hidden layer 
#dnnModel.add(layers.Dense(60, activation='relu'))

# Layer 3 = hidden layer 
#dnnModel.add(layers.Dense(30, activation='linear'))

# Layer 4 = output layer
dnnModel.add(layers.Dense(len(category2idx), activation='sigmoid'))

dnnModel.summary()
dnnModel.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'] )
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
y = data.pop('category_idx')
X = data

import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = X.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

X_train,X_test,y_train,y_test = train_test_split(X.index,y.index,test_size=0.2)
xtrain=X.iloc[X_train]
xtest=X.iloc[X_test]
y_train=y.iloc[y_train]
y_test=y.iloc[y_test]

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from collections import Counter
print('Before Resampled dataset shape' )
print( y_train.shape)
sm = RandomOverSampler('all',random_state=420)
xtrain, y_train = sm.fit_resample(xtrain, y_train)
print(' Resampled dataset shape' )
print( y_train.shape)

#smote = SMOTE()
#xtrain, y_train = smote.fit_resample(xtrain, y_train)
#print(' Resampled dataset shape2' )
#print( y_train.shape)

ytrain = np.zeros((len(y_train), len(idx2category.values())))
ytest = np.zeros((len(y_test), len(idx2category.values())))
for i in range(len(y_train)):
    #print(y_train.values[i])
    ytrain[i][y_train.values[i]]=1
    
for i in range(len(y_test)):
    #print(y_train.values[i])
    ytrain[i][y_test.values[i]]=1

xtrain=tf.convert_to_tensor(xtrain)

ytraintens=tf.convert_to_tensor(ytrain)

h  = dnnModel.fit( xtrain, ytraintens, epochs=25, batch_size=5)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                1620      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1830      
Total params: 3,450
Trainable params: 3,450
Non-trainable params: 0
_________________________________________________________________
Before Resampled dataset shape
(18936,)
 Resampled dataset shape
(152656,)


C:\Users\kalus\anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=all as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/25
30532/30532 [==============================] - 27s 872us/step - loss: 2.4035 - accuracy: 0.4200
Epoch 2/25
30532/30532 [==============================] - 25s 823us/step - loss: 2.0821 - accuracy: 0.4822
Epoch 3/25
30532/30532 [==============================] - 25s 823us/step - loss: 1.9911 - accuracy: 0.4982
Epoch 4/25
30532/30532 [==============================] - 26s 842us/step - loss: 1.9286 - accuracy: 0.5128
Epoch 5/25
30532/30532 [==============================] - 25s 823us/step - loss: 1.8887 - accuracy: 0.5209
Epoch 6/25
30532/30532 [==============================] - 26s 850us/step - loss: 1.8623 - accuracy: 0.5244
Epoch 7/25
30532/30532 [==============================] - 25s 811us/step - loss: 1.8460 - accuracy: 0.5273
Epoch 8/25
30532/30532 [==============================] - 25s 826us/step - loss: 1.8321 - accuracy: 0.5290
Epoch 9/25
30532/30532 [==============================] - 25s 808us/step - loss: 1.8215 - accuracy: 0.5310
Epoch 10/25
30532/30532 [============